In [1]:
import torch
from torch.utils.data import DataLoader
import mlflow
from time import time
import hiddenlayer as HL
import os
import cv2

# i made all of these !
from models import Backbone as Model
from video_loader import vidSet
from utils import Params, count_parameters
from loss import ConstrastiveLoss


In [2]:
# we want a class for our parameters because it is wayyyy easier to log them this way 
args = Params(10, 1, 0.00005)


In [3]:
!nvidia-smi

# for running in command line 
# watch -n 0.5 nvidia-smi

Fri Jun 26 10:38:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:03:00.0 Off |                  N/A |
| 28%   30C    P8    23W / 250W |   1271MiB / 12066MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:83:00.0 Off |                    0 |
| N/A   32C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [4]:
device = 'cuda:2'

In [5]:
videos_path = '/share/lazy/will/ConstrastiveLoss/bdd100k/videos/New'

path_list = []
for (dirpath, _, filenames) in os.walk(videos_path):
    for filename in filenames:
        path_list.append(os.path.abspath(os.path.join(videos_path, filename)))

# takes a list of file paths to .mp4s and returns a dataloader ov the frames
vidset_train = vidSet(path_list[:100])

vidloader_train = DataLoader(vidset_train, batch_size=args.batch_size, shuffle=False, pin_memory=True)


In [ ]:
import matplotlib.pyplot as plt

for i in range(4):
    plt.imshow(next(iter(vidloader_train))[i].permute(1,2,0).int())
    plt.show()

In [6]:
# set the file path where the logs will be stored. this should be a global reference since many different scripts will reference it from different directories
mlflow.tracking.set_tracking_uri('file:/share/lazy/will/ConstrastiveLoss/Logs')

# a new experiment will be created if one by that name does not already exists
mlflow.set_experiment('Constrastive loss unsupervised')


def train(model, train_data, loss, optimizer):

    for epoch in range(args.epochs):
        t0 = time()
        train_loss = 0

        for batch_idx, img_tensor in enumerate(train_data):

            # select FOUR images total, two pos two neg. the forward pass size is 1 gb for the overparameterized model so it is important
            # to pick a combination that fits into memory. 

            # Select 2 positives, or the first 2 frames
            positives_tensor = img_tensor[:2].to(device)

            # this I THINK selects the last two images in a sequence, loss does not count observations of the same class, so it is ok that they are similar
            # however it may be slower to learn.
            negatives_tensor = img_tensor[3:5].to(device)

            # set parameter gradients to zero 
            optimizer.zero_grad()

            # Forward pass: compute the output
            positive_latent_tensor = model(positives_tensor)
            negative_latent_tensor = model(negatives_tensor)

            # Computation of the cost J
            cost = loss(positive_latent_tensor, negative_latent_tensor)  

            # Backward propagation
            cost.backward()  # <= compute the gradients
            train_loss += cost.item()

            # Update parameters (weights and biais)
            optimizer.step()

            # Basically like having a batch size of 2, hoping this helps with the data transfer bottleneck since 
            # i can have the data loader doing stuff in the background with pin_memory, more time/compute per batch
            positives_tensor = img_tensor[6:8].to(device)        
            negatives_tensor = img_tensor[9:11].to(device)
            
            # Forward pass: compute the output
            positive_latent_tensor = model(positives_tensor)
            negative_latent_tensor = model(negatives_tensor)

            cost = loss(positive_latent_tensor, negative_latent_tensor)
            cost.backward()
            
            optimizer.step()
            
            train_loss += cost.item()
        t1 = time()

        
        ret = {'Train Loss':train_loss, 'Epoch time':t1-t0}
        yield ret.items()


In [ ]:
# load the model
from models import Backbone as Model

model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)
loss = ConstrastiveLoss(device=device)

run_name = 'Ten videos'
with mlflow.start_run(run_name = run_name) as run:
    for key, value in vars(args).items():
        mlflow.log_param(key, value)

    mlflow.log_param('Parameters', count_parameters(model))

    for epoch, items in enumerate(train(model, vidloader_train, loss, optimizer)):
        for key, value in items:
            print(key, value)
            mlflow.log_metric(key, value, epoch)

    torch.save({
        'model':model.state_dict(),
        'optimizer':optimizer.state_dict(),
        }, 'run_stats.pyt')
    mlflow.log_artifact('run_stats.pyt')

    torch.cuda.empty_cache()

    # save an architecture diagram
    HL.transforms.Fold("Conv > BatchNorm > Relu", "ConvBnRelu"),
    HL.build_graph(model, torch.zeros([args.batch_size, 3, 288, 512]).to(device)).save('architecture', format='png')
    mlflow.log_artifact('architecture.png')
    

2020/06/26 10:39:06 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: [Errno 11] Resource temporarily unavailable
